In [1]:
!pip list | grep -E "awscli|boto3|botocore"

awscli                        1.27.74
boto3                         1.26.162
botocore                      1.29.162


In [2]:
from sagemaker import get_execution_role

In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMakerServiceCatalogProductsUseRole


In [4]:
#!wget https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
#!unzip bedrock-python-sdk.zip -d bedrock-sdk
#!rm -rf bedrock-python-sdk.zip

In [5]:
install_needed = False

In [6]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-1.29.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-1.26.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-1.27.162-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

In [7]:
import boto3
import langchain

In [8]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.249
boto3 version check: 1.26.162


In [9]:
bedrock_region = "us-west-2" 

In [10]:
is_internal_use = True # 내부 직원 용
# is_internal_use = False # 고객 용

In [11]:
bedrock_region = "us-west-2" 
bedrock_region = "us-west-2" 
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

In [12]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [13]:
output_text = bedrock_client.list_foundation_models()
output_text 

{'ResponseMetadata': {'RequestId': 'bd5253c8-b631-41a3-a89e-95a802d0783a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 17 Aug 2023 23:20:43 GMT',
   'content-type': 'application/json',
   'content-length': '256',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bd5253c8-b631-41a3-a89e-95a802d0783a'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'}]}

In [14]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [15]:
import json
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

In [16]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get('results')[0].get('outputText')
print('output: ', outputText)

output:   "Jane Doe"
Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service experience you had when you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we fell short of your expectations. We take feedback like yours very seriously, and we are committed to resolving the issue and ensuring that we provide better service in the future.

If you would like to discuss this matter further, please do not hesitate to contact me directly. We value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager


In [17]:
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            output.append(text)
            print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
            i+=1

		**Chunk 1**
 "Jane Doe"
Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service experience you had when

		**Chunk 2**
 you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we fell short of your expectations. We take feedback like yours very seriously, and we are committed to resolving the issue and ensuring that we provide better service in the future.

If

		**Chunk 3**
 you would like to discuss this matter further, please do not hesitate to contact me directly. We value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager



In [18]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**

 "Jane Doe"
Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service experience you had when you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we fell short of your expectations. We take feedback like yours very seriously, and we are committed to resolving the issue and ensuring that we provide better service in the future.

If you would like to discuss this matter further, please do not hesitate to contact me directly. We value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager


## Bedrock streaming using LLM

[bedrock stream reference](https://github.com/langchain-ai/langchain/issues/9094)

[Open AI stream reference](https://python.langchain.com/docs/modules/model_io/models/chat/streaming)

[InvokeModelWithResponseStream](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_InvokeModelWithResponseStream.html)

In [20]:
import json
from typing import Generator

def process_prompt_streaming(prompt) -> Generator[str, None, None]:
    cont = True
    prompt_text = prompt + '\n'
    while cont:
        response_stream = bedrock_client.invoke_model_with_response_stream(
            body=json.dumps({'inputText': prompt_text}),
            modelId='amazon.titan-tg1-large',
            accept='application/json',
            contentType='application/json'
        )
        status_code = response_stream['ResponseMetadata']['HTTPStatusCode']
        if status_code != 200:
            raise ValueError(
                f"Error invoking Bedrock API: {status_code}"
            )
        for response in response_stream['body']:
            json_response = json.loads(response['chunk']['bytes'])
            yield json_response['outputText']
            if json_response['completionReason'] == 'FINISH':
                cont = False
                break

for m in process_prompt_streaming('Command: Tell me a story about a tortoise and a hare.'):
    print(m)

Once upon a time, there was a tortoise and a hare who were good friends. One day, the tortoise challenged the hare to a race. The hare accepted the challenge 
thinking that it will be an easy win. So, when the race started, the hare ran very fast leaving the tortoise way behind. He reached the finish line while the tortoise was still a long way behind. He looked back and saw that the tortoise was not far behind. So, he thought to himself that the tortoise will not be able to reach the finish line before he does. So, he took a nap.
The tortoise and the hare had been friends for a long time. One day, they decided to have a race. The hare expected to win easily because the tortoise was moving 
so slowly. So he took a nap while the tortoise kept going. When the hare woke up, the tortoise was already at the finish line. To his dismay, the tortoise won the race while he was sleeping.


[Implementing Locally-Hosted Llama2 Chat UI Using Streamlit](https://medium.com/@daydreamersjp/implementing-locally-hosted-llama2-chat-ui-using-streamlit-53b181651b4e)

In [21]:
from langchain.schema import (
    HumanMessage,
)

In [22]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
def select_llm()
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    return bedrock_client.invoke_model_with_response_stream(
        body=json.dumps({'inputText': prompt_text}),
        modelId='amazon.titan-tg1-large',
        accept='application/json',
        contentType='application/json'
    )

# (2023-08-18) 현재 기준으로 아직 Bedrock stream을 Langchain을 제공할 수 없음.